In [1]:
import pandas as pd
import numpy as np

In [2]:
df_teamstats = pd.read_csv("data/AmericanFootball98.csv")
df_sb = pd.read_csv("data/superbowl.csv")

In [3]:
df_teamstats.drop(df_teamstats.columns[33:], axis=1, inplace=True)
df_sb.drop(df_sb.columns[6:], axis=1, inplace=True)
df_sb.drop(0, inplace=True)

#changes date to year
df_sb['Year'] = pd.to_datetime(df_sb['Date']).dt.year
df_sb.drop(labels = 'Date', axis=1, inplace=True)

#changes sb roman number to just int
df_sb['SB'] = df_sb['SB'].str.replace(r'[a-zA-Z]', '', regex=True)
df_sb['SB'] = df_sb['SB'].str.replace(r'\(|\)', '', regex=True)

#moves year
column_to_move = df_sb.pop("Year")
df_sb.insert(0, "Year", column_to_move)

In [4]:
#Extracting the team names and years from df_teamstats
df_teamstats['Team'] = df_teamstats['team_code'].str.extract(r'([A-Za-z]{3})')
df_teamstats['Year'] = df_teamstats['team_code'].str.extract(r'(\d{4})')

#reorder columns
column_to_move = df_teamstats.pop("Year")
df_teamstats.insert(0, "Year", column_to_move)
column_to_move = df_teamstats.pop("Team")
df_teamstats.insert(1, "Team", column_to_move)

#delete team_code
df_teamstats = df_teamstats.drop('team_code', axis=1)
df_teamstats

,Year,Team,wins,losses,PF,yards,plays,yards/play,TO,Fumbles Lost,...,pen yards,pen 1st downs,number drives,score percentage,turnover percentage,avg start,avg time per drive,avg plays per drive,avg yards per drive,avg points per drive
0,2019,nwe,12,4,420,5664,1095,5.2,15,6,...,828,31,185,36.8,7.6,Own 32.5,2:40,5.97,29.9,1.99
1,2019,buf,10,6,314,5283,1018,5.2,19,7,...,927,32,183,30.6,10.4,Own 28.9,2:34,5.62,28.6,1.66
2,2019,nyj,7,9,276,4368,956,4.6,25,9,...,1105,30,183,23.0,11.5,Own 26.4,2:28,5.23,23.1,1.21
3,2019,mia,5,11,306,4960,1022,4.9,26,8,...,769,41,180,30.6,13.3,Own 27.6,2:33,5.78,27.1,1.63
4,2019,rav,14,2,531,6521,1064,6.1,15,7,...,867,27,163,52.1,8.6,Own 28.9,3:22,6.61,39.3,2.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664,1999,ram,13,3,526,6412,994,6.5,31,16,...,889,26,188,39.9,15.4,Own 31.5,2:42,5.44,34.1,2.37
665,1999,car,8,8,421,5686,982,5.8,34,19,...,857,21,192,38.0,15.1,Own 32.7,2:27,5.26,29.6,2.14
666,1999,atl,5,11,285,4542,931,4.9,35,16,...,968,26,184,25.0,17.4,Own 30.2,2:30,5.17,24.6,1.42
667,1999,sfo,4,12,295,5380,1012,5.3,32,13,...,1045,26,182,26.9,15.9,Own 28.5,2:40,5.69,29.5,1.42


In [5]:
#Dictionaries
sb_dict = {
    "New England Patriots":"Patriots",
    "Miami Dolphins":"Dolphins",
    "Buffalo Bills":"Bills",
    "New York Jets":"Jets",
    "Cincinnati Bengals":"Bengals",
    "Pittsburgh Steelers":"Steelers",
    "Baltimore Ravens":"Ravens",
    "Cleveland Browns": "Browns", #team has not been to the SB yet
    "Baltimore Colts":"Colts",
    "Indianapolis Colts":"Colts", #2nd Colts Team
    "Jacksonville Jaguars":"Jaguars", #team has not been to the SB yet
    "Tennessee Titans":"Titans", 
    "Houston Texans":"Texans", #team has not been to the SB yet
    "Denver Broncos":"Broncos",
    "Kansas City Chiefs":"Chiefs",
    "San Diego Chargers":"Chargers", #omitted LA Chargers, no SB appearance
    "Oakland Raiders":"Raiders", #omitted LV Raiders, no SB appearance
    "Los Angeles Raiders":"Raiders", #2nd Raiders Team
    "New York Giants":"Giants",
    "Washington Redskins":"Redskins", #omitted Commanders, no SB appearance
    "Dallas Cowboys":"Cowboys",
    "Philadelphia Eagles":"Eagles",
    "Chicago Bears":"Bears",
    "Minnesota Vikings":"Vikings",
    "Detroit Lions":"Lions", #team has not been to SB yet
    "Green Bay Packers":"Packers",
    "Tampa Bay Buccaneers":"Buccaneers",
    "Carolina Panthers":"Panthers",
    "Atlanta Falcons":"Falcons",
    "New Orleans Saints":"Saints",
    "Seattle Seahawks":"Seahawks",
    "St. Louis Rams":"Rams",
    "Los Angeles Rams":"Rams", #2nd Rams Team
    "Arizona Cardinals":"Cardinals",
    "San Francisco 49ers":"49ers"
}

team_dict = {
    "nwe": "Patriots",
    "mia":"Dolphins",
    "buf":"Bills",
    "nyj":"Jets",
    "cin":"Bengals",
    "pit":"Steelers",
    "rav":"Ravens",
    "cle":"Browns",
    "clt":"Colts",
    "jax":"Jaguars",
    "oti":"Titans",
    "htx":"Texans",
    "den":"Broncos",
    "kan":"Chiefs",
    "sdg":"Chargers",
    "rai":"Raiders",
    "nyg":"Giants",
    "was":"Redskins",
    "dal":"Cowboys",
    "phi":"Eagles",
    "chi":"Bears",
    "min":"Vikings",
    "det":"Lions",
    "gnb":"Packers",
    "tam":"Buccaneers",
    "car":"Panthers",
    "atl":"Falcons",
    "nor":"Saints",
    "sea":"Seahawks",
    "ram":"Rams",
    "crd":"Cardinals",
    "sfo":"49ers"
}

In [6]:
#changes team names to mascots
df_sb['Winner'] = df_sb['Winner'].replace(sb_dict)
df_sb['Loser'] = df_sb['Loser'].replace(sb_dict)

df_teamstats['Team'] = df_teamstats['Team'].replace(team_dict)

df_sb.head(10)

,Year,SB,Winner,Winner Pts,Loser,Loser Pts
1,2019,53,Patriots,13,Rams,3
2,2018,52,Eagles,41,Patriots,33
3,2017,51,Patriots,34,Falcons,28
4,2016,50,Broncos,24,Panthers,10
5,2015,49,Patriots,28,Seahawks,24
6,2014,48,Seahawks,43,Broncos,8
7,2013,47,Ravens,34,49ers,31
8,2012,46,Giants,21,Patriots,17
9,2011,45,Packers,31,Steelers,25
10,2010,44,Saints,31,Colts,17


In [7]:
df_teamstats.head(10)

,Year,Team,wins,losses,PF,yards,plays,yards/play,TO,Fumbles Lost,...,pen yards,pen 1st downs,number drives,score percentage,turnover percentage,avg start,avg time per drive,avg plays per drive,avg yards per drive,avg points per drive
0,2019,Patriots,12,4,420,5664,1095,5.2,15,6,...,828,31,185,36.8,7.6,Own 32.5,2:40,5.97,29.9,1.99
1,2019,Bills,10,6,314,5283,1018,5.2,19,7,...,927,32,183,30.6,10.4,Own 28.9,2:34,5.62,28.6,1.66
2,2019,Jets,7,9,276,4368,956,4.6,25,9,...,1105,30,183,23.0,11.5,Own 26.4,2:28,5.23,23.1,1.21
3,2019,Dolphins,5,11,306,4960,1022,4.9,26,8,...,769,41,180,30.6,13.3,Own 27.6,2:33,5.78,27.1,1.63
4,2019,Ravens,14,2,531,6521,1064,6.1,15,7,...,867,27,163,52.1,8.6,Own 28.9,3:22,6.61,39.3,2.96
5,2019,Steelers,8,8,289,4428,937,4.7,30,11,...,893,43,185,28.6,15.7,Own 29.4,2:28,5.17,23.2,1.38
6,2019,Browns,6,10,335,5455,973,5.6,28,7,...,1106,35,182,34.1,14.8,Own 29.3,2:36,5.47,29.7,1.84
7,2019,Bengals,2,14,279,5169,1049,4.9,30,14,...,761,36,175,30.3,16.0,Own 27.8,2:38,5.97,29.0,1.53
8,2019,Texans,10,6,378,5792,1017,5.7,22,8,...,892,31,167,37.7,12.0,Own 28.6,2:50,6.15,34.3,2.19
9,2019,Titans,9,7,402,5805,949,6.1,17,9,...,932,36,183,31.7,8.2,Own 27.5,2:28,5.24,31.5,2.04
